In [38]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none
pd.set_option('display.max_columns', None)

### I will create the interaction matrix from history data and save it in order to quick access

In [39]:
size = 'demo'
type_ = 'validation'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/{type_}/history.parquet'

history_df = pd.read_parquet(history_train_path)
print(history_df.shape)

behaviors_path = f'./files/parquet/ebnerd_{size}/{type_}/behaviors.parquet'
behaviors_df = pd.read_parquet(behaviors_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_df.shape)


(1562, 5)
Raw demo validation behaviors df shape:           (25356, 17)


In [40]:
print(history_df.explode('impression_time_fixed').impression_time_fixed.min())
print(history_df.explode('impression_time_fixed').impression_time_fixed.max())
history_df.head(2)

2023-05-04 07:00:11
2023-05-25 06:59:54


,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,750497,"[2023-05-04T09:42:39.000000, 2023-05-04T09:43:...","[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[2023-05-04T07:53:42.000000, 2023-05-04T15:59:...","[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."


In [41]:
print(behaviors_df.impression_time.min())
print(behaviors_df.impression_time.max())
behaviors_df.head(2)

2023-05-25 07:00:15
2023-06-01 06:59:33


,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,144772,NaN,2023-05-30 14:21:34,29.0,NaN,2,"[9788239, 9780702, 9553264, 9787499, 6741781, ...",[9783042],76658,False,NaN,NaN,NaN,False,29,7.0,59.0
1,144777,NaN,2023-05-30 14:22:11,10.0,NaN,2,"[9788521, 9786217, 9553264, 9788361, 9788352, ...",[9788125],76658,False,NaN,NaN,NaN,False,29,58.0,98.0


In [42]:
# Preprocess behaviour
behaviors_df = behaviors_df[['article_ids_clicked','user_id','read_time']]
behaviors_df = behaviors_df.explode('article_ids_clicked')


In [43]:
behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()


,user_id,article_ids_clicked
0,19181,"[9783019, 9770145, 9782519, 9782899, 9788947, ..."
1,21271,"[9785113, 9339920, 9786222, 9782407, 9784097, ..."
2,21774,[9785835]
3,22779,"[9787261, 9784852, 9780702, 9785111, 9782806, ..."
4,22895,"[9782996, 9790574, 9790475, 9781013, 9780514, ..."
...,...,...
1557,2581162,"[9781998, 9695098, 9782487, 9780467, 9783404, ..."
1558,2583035,"[9782108, 9781906, 9785835, 9780439, 9783057, ..."
1559,2583477,"[9780210, 9781057, 9786204, 9777846, 9787469, ..."
1560,2584367,"[9785434, 9785111, 9772508]"


In [44]:
behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index()


,user_id,read_time
0,19181,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,..."
1,21271,"[23.0, 30.0, 15.0, 16.0, 83.0, 10.0, 25.0]"
2,21774,[47.0]
3,22779,"[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0..."
4,22895,"[14.0, 8.0, 29.0, 17.0, 17.0, 9.0, 8.0, 14.0, ..."
...,...,...
1557,2581162,"[52.0, 20.0, 34.0, 16.0, 104.0, 22.0, 52.0, 83..."
1558,2583035,"[12.0, 20.0, 15.0, 22.0, 156.0, 8.0, 23.0, 8.0..."
1559,2583477,"[76.0, 38.0, 411.0, 15.0, 28.0, 19.0, 22.0, 35.0]"
1560,2584367,"[44.0, 110.0, 51.0]"


In [45]:
behaviors_df= pd.merge(behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index(),behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index(),on='user_id')

In [46]:
behaviors_df.head(2)

,user_id,article_ids_clicked,read_time
0,19181,"[9783019, 9770145, 9782519, 9782899, 9788947, ...","[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,..."
1,21271,"[9785113, 9339920, 9786222, 9782407, 9784097, ...","[23.0, 30.0, 15.0, 16.0, 83.0, 10.0, 25.0]"


In [47]:
try:
    history_df.drop('impression_time_fixed', axis=1, inplace=True) # i will drop this column initialy even though i can proccess it and use it
except:
    pass

In [48]:
history_df['scroll_percentage_fixed_len'] = history_df['scroll_percentage_fixed'].apply(lambda lst: len(lst))
history_df['article_id_fixed_len'] = history_df['article_id_fixed'].apply(lambda lst: len(lst))
history_df['read_time_fixed_len'] = history_df['read_time_fixed'].apply(lambda lst: len(lst))

In [49]:
history_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64...",409,409,409
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18....",393,393,393


### The lenths of the lists in each column seem equal

In [50]:
history_df['scroll_percentage_fixed_has_0_in'] = history_df['scroll_percentage_fixed'].apply(lambda lst: 0 in lst)
history_df['read_time_fixed_has_0_in'] = history_df['read_time_fixed'].apply(lambda lst: 0 in lst)
history_df['read_time_fixed_has_nan'] = history_df['read_time_fixed'].apply(lambda lst: pd.isna(lst.any()))

In [51]:
history_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,scroll_percentage_fixed_len,article_id_fixed_len,read_time_fixed_len,scroll_percentage_fixed_has_0_in,read_time_fixed_has_0_in,read_time_fixed_has_nan
0,750497,"[100.0, 65.0, 100.0, 100.0, 100.0, 100.0, 100....","[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64...",409,409,409,False,True,False
1,22779,"[52.0, 39.0, 62.0, 38.0, 74.0, 19.0, 30.0, 56....","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18....",393,393,393,False,True,False
2,373598,"[nan, nan, nan, 59.0, 33.0, 75.0, nan, nan, 76...","[9514481, 9514481, 9111040, 9750389, 9750307, ...","[0.0, 0.0, 0.0, 3.0, 9.0, 117.0, 39.0, 0.0, 8....",233,233,233,False,True,False
3,383378,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[9747490, 9749036, 9749025, 9748792, 9748592, ...","[85.0, 18.0, 133.0, 191.0, 331.0, 56.0, 43.0, ...",664,664,664,False,True,False
4,411733,"[20.0, 14.0, 61.0, 55.0, 21.0, 81.0, 100.0, 10...","[9750081, 9750111, 9750039, 9749948, 9749729, ...","[2.0, 4.0, 6.0, 9.0, 1.0, 30.0, 37.0, 5.0, 3.0...",281,281,281,False,True,False


In [52]:
history_df['read_time_fixed_has_nan'].unique()

array([False])

### There both nulls and zeros in the scroll_percentage_fixed column but not in read_time_fixed, i will use the read_time_fixed for interraction matrix or i will have to fix the nans to scroll_percentage_fixed

In [53]:
# # Combine interactions into a single metric (Potential use of scroll_percentage_fixed column)
# def combine_interactions(interactions, interactions2, w1=0.5, w2=0.5):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# history_dfraction_combined'] = history_df(lambda row:   combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed']) , axis=1)

# history_df ['scroll_percentage_fixed', 'read_time_fixed'], axis=1, inplace=True)

In [54]:
history_df.drop(['scroll_percentage_fixed_len', 'article_id_fixed_len', 'read_time_fixed_len', 'scroll_percentage_fixed_has_0_in', 'read_time_fixed_has_nan', 'scroll_percentage_fixed','read_time_fixed_has_0_in'], axis=1, inplace=True)

In [55]:
history_df.head(2)

,user_id,article_id_fixed,read_time_fixed
0,750497,"[9749224, 9749156, 9749224, 9748948, 9748980, ...","[49.0, 5.0, 7.0, 151.0, 214.0, 199.0, 22.0, 64..."
1,22779,"[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."


In [56]:
history_df.shape

(1562, 3)

In [57]:
## Merge behaviours and history-left join
final_df = pd.merge(behaviors_df,history_df,on='user_id',how='left')
final_df.shape

(1562, 5)

In [58]:
final_df.head()

,user_id,article_ids_clicked,read_time,article_id_fixed,read_time_fixed
0,19181,"[9783019, 9770145, 9782519, 9782899, 9788947, ...","[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...","[9749036, 9749092, 9749156, 9748916, 9747959, ...","[48.0, 2.0, 2.0, 10.0, 6.0, 14.0, 4.0, 2.0, 3...."
1,21271,"[9785113, 9339920, 9786222, 9782407, 9784097, ...","[23.0, 30.0, 15.0, 16.0, 83.0, 10.0, 25.0]","[9749601, 9749601, 9752375, 9752479, 9745405, ...","[42.0, 1.0, 11.0, 39.0, 7.0, 2.0, 31.0, 18.0, ..."
2,21774,[9785835],[47.0],"[9748041, 9750718, 9747859, 9747411, 9750189, ...","[114.0, 6.0, 28.0, 109.0, 33.0, 74.0, 139.0, 5..."
3,22779,"[9787261, 9784852, 9780702, 9785111, 9782806, ...","[14.0, 58.0, 11.0, 14.0, 10.0, 11.0, 9.0, 61.0...","[9749025, 9750090, 9750015, 9750161, 9745750, ...","[4.0, 16.0, 2.0, 9.0, 40.0, 7.0, 9.0, 8.0, 18...."
4,22895,"[9782996, 9790574, 9790475, 9781013, 9780514, ...","[14.0, 8.0, 29.0, 17.0, 17.0, 9.0, 8.0, 14.0, ...","[9750397, 9750074, 9750296, 9745750, 9750296, ...","[5.0, 3.0, 34.0, 7.0, 73.0, 0.0, 7.0, 43.0, 2...."


In [59]:
def concatenate_lists(df, col1, col2, new_col_name):
    df[new_col_name] = df.apply(lambda row: row[col1] + row[col2] if isinstance(row[col1], list) and isinstance(row[col2], list) else row[col1] if isinstance(row[col1], list) else row[col2] if isinstance(row[col2], list) else [], axis=1)
    return df

In [60]:
final_df = concatenate_lists(final_df,"article_ids_clicked","article_id_fixed","merge_article_ids")
final_df = concatenate_lists(final_df,"read_time","read_time_fixed","merge_read_time")

In [61]:
final_df.shape

(1562, 7)

In [62]:
final_df = final_df[['merge_article_ids','merge_read_time','user_id']]

In [63]:
final_df.head(2)

,merge_article_ids,merge_read_time,user_id
0,"[9783019, 9770145, 9782519, 9782899, 9788947, ...","[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...",19181
1,"[9785113, 9339920, 9786222, 9782407, 9784097, ...","[23.0, 30.0, 15.0, 16.0, 83.0, 10.0, 25.0]",21271


In [64]:
# final_df.set_index('user_id',inplace=True)

In [65]:
final_df = final_df.explode('merge_article_ids')

In [66]:
final_df.shape

(25505, 3)

In [67]:
final_df.head()

,merge_article_ids,merge_read_time,user_id
0,9783019,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...",19181
0,9770145,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...",19181
0,9782519,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...",19181
0,9782899,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...",19181
0,9788947,"[15.0, 6.0, 5.0, 21.0, 57.0, 18.0, 19.0, 13.0,...",19181


In [68]:
# Creation of interaction matrix

interaction_matrix = pd.DataFrame(columns=final_df.columns)
counter = 0
for user_id in tqdm(np.unique(final_df.index.values).tolist()):
    
    temp_df = final_df[final_df.index == user_id].copy()
    temp_df = temp_df.explode('merge_article_ids')
    temp_df.set_index('user_id',inplace=True)
    temp_lst = []

    # Fix the lists that exist in each cell and keep the values that i care
    temp_counter = 0
    for lst in list(temp_df['merge_read_time']):
        temp_lst.append(lst[temp_counter])
        temp_counter += 1
    
    temp_df['merge_read_time_fixed'] = temp_lst

    # Drop first unneccesary column
    temp_df.drop('merge_read_time', axis=1, inplace= True)
    temp_pivoted_df = temp_df.pivot_table(index = temp_df.index, columns='merge_article_ids', values='merge_read_time_fixed', aggfunc='sum').copy()

    if counter == 0:
        interaction_matrix = temp_pivoted_df.copy()
        #print(interaction_matrix)
    else:
        interaction_matrix = pd.concat([interaction_matrix, temp_pivoted_df], axis=0, ignore_index=False)
    
    counter += 1

C:\Users\Intel PC G5900\AppData\Local\Temp\ipykernel_4788\1750402660.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id in tqdm(np.unique(final_df.index.values).tolist()):


  0%|          | 0/1562 [00:00<?, ?it/s]

In [69]:
interaction_matrix.shape

(1562, 1144)

In [70]:
interaction_matrix.head()

merge_article_ids,6741781,9052390,9277736,9770145,9777199,9778795,9780267,9780547,9780677,9782407,9782423,9782519,9782899,9783019,9783137,9783164,9783655,9784575,9784642,9785596,9786359,9787593,9787863,9788188,9788666,9788947,9789702,9339920,9784097,9785113,9785668,9786172,9786222,9785835,9780702,9782806,9784044,9784344,9784852,9785111,9786209,9787261,9774526,9777705,9778796,9779615,9780329,9780514,9781013,9781257,9781667,9781709,9782092,9782126,9782361,9782421,9782709,9782915,9782996,9783646,9783852,9784393,9784489,9784607,9784696,9784805,9784951,9785019,9785174,9785604,9785665,9785742,9785790,9785888,9785892,9786213,9786729,9788766,9790475,9790574,9790825,9717946,9766514,9779092,9779411,9780439,9780736,9780861,9781523,9781621,9781785,9781838,9782487,9782652,9782808,9783236,9784056,9784870,9785593,9788151,9788197,9788862,9789037,9789704,9789910,9789922,9789974,9790002,9790521,9790572,9790700,9773779,9779541,9779777,9780020,9780447,9782005,9783404,9786693,9790539,9780374,9780815,9781598,9781624,9781983,9781998,9782282,9782635,9783314,9784064,9785016,9785404,9785829,9785899,9786245,9786279,9786549,9786567,9787659,9788106,9788323,9788393,9788411,9788576,9788898,9789065,9789418,9789471,9789537,9789692,9789832,9790682,9779724,9780476,9782180,9785680,9786288,9786860,9786932,9789642,9790548,9772508,9780283,9782085,9782884,9783237,9784275,9784559,9785471,9785553,9785828,9786313,9786566,9787465,9787487,9783732,9788183,9788557,9366571,9781001,9785868,9785923,9785992,9779122,9783276,9783296,9784425,9785049,9785107,9785205,9787586,7594265,9777016,9779066,9780280,9780372,9781316,9781354,9781389,9781535,9781855,9781991,9782239,9782378,9784444,9784947,9785017,9785020,9785062,9785973,9786204,9788524,9789125,9789494,9789997,9790811,9790822,9790885,9790886,9782998,9781796,9781875,9786176,9786312,9788521,9789647,9789866,9779650,9782205,9779659,9782103,9786139,9786210,9787768,9788125,9781017,9788108,9788352,9789386,9789728,9788361,9789493,9784662,9785454,9786268,9781856,9783334,9754241,9779653,8380569,9448302,9531110,9577133,9776147,9779365,9780498,9780517,9781086,9781520,9781878,9782115,9782142,9782616,9782670,9782672,9782695,9782726,9783057,9783853,9783858,9784430,9784506,9784591,9784679,9785645,9785986,9786381,9786699,9787501,9787739,9788132,9788184,9788471,9788692,9789001,9789018,9789481,9789623,9790091,9790272,9790559,9790752,9788907,9789747,9749014,9769917,9779867,9780986,9782929,9783865,9783993,9784856,9786560,9786649,9788038,9788067,9788134,9788797,9780604,9783405,9784879,9787767,9789745,9790360,9790406,9777858,9779205,9779263,9779737,9780195,9782722,9782809,9783183,9783278,9767830,9781814,9782769,9783075,9783977,9786247,9780210,9780644,9780925,9782108,9783213,9783850,9785487,9787022,9788849,9789096,9756075,9760046,9769155,9778939,9779648,9779738,9781262,9781636,9782027,9782656,9782993,9784138,9784228,9784952,9785424,9785799,9786497,9787185,9787553,9788552,9788950,9789810,9790052,9790121,9790278,9790293,9784804,9778869,9779807,9782355,9782360,9784793,9789475,9789502,9789911,9695098,9771113,9771170,9775978,9780860,9782219,9782391,9782449,9782837,9782879,9783740,9784281,9784702,9784939,9785475,9786618,9787332,9787769,9788043,9788045,9788466,9789831,9790019,9790040,9790645,9782237,9783509,9785500,9788736,9788841,9782517,9783803,9783990,9786495,9777983,9782057,9786468,9787855,9788794,9789710,9787499,9784710,9787524,9788149,9788621,9788917,9788203,9782181,9787469,9776322,9781424,9782800,9785718,9785732,9786906,9788705,9779538,9781057,9787811,9121218,9777026,9778971,9780968,9783004,9783042,9784058,9784160,9786524,9787679,9783628,9277339,9476171,9726237,9777950,9779294,9780284,9780300,9780718,9781763,9781913,9781932,9782021,9782046,9782133,9782202,9782234,9782248,9782321,9782418,9783024,9783159,9784214,9785009,9785076,9785499,9786159,9786378,9786563,9787243,9787441,9788547,9789539,9789600,9790476,9790532,8496358,9779225,9782633,9785209,9787264,9789754,9790414,9783203,9783952,9786306,9787176,9775964,9777397,9778732,9778944,9780482,9781506,978319

In [71]:
fillna_value = '0'
interaction_matrix.fillna(0, inplace=True)

In [72]:
# Save the interaction matrix to pickle
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'

interaction_matrix.to_pickle(interaction_matrix_file_path)

In [45]:
# Read interaction matrix pickle file
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)

FileNotFoundError: [Errno 2] No such file or directory: './files/pickle/interaction_matrix_demo_0.pkl'

## Other

In [ ]:
# # Combine interactions into a single metric
# def combine_interactions(interactions, interactions2, w1=1, w2=1):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# # Flatten the interaction data with combined interactions
# flattened_data = []

# for _, row in history_df.iterrows():
#     user_id = row['user_id']
#     article_ids = row['article_id_fixed']
#     combined_interactions = combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed'])
    
#     for article_id, interaction_combined in zip(article_ids, combined_interactions):
#         flattened_data.append({
#             'user_id': user_id, 
#             'article_id_fixed': article_id, 
#             'interaction_combined': interaction_combined
#         })

# # Convert to DataFrame
# interaction_df = pd.DataFrame(flattened_data)